# day 8

https://adventofcode.com/8/day/8

In [ ]:
import itertools
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day08.txt')

LOGGER = logging.getLogger('day08')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
def parse_data(s: str):
    return {j + i * 1j: c
            for (i, line) in enumerate(s.strip().split('\n'))
            for (j, c) in enumerate(line)}

#### function def

In [ ]:
def build_char_map(d: dict[complex, str]) -> dict[str, list[complex]]:
    cm = {}
    for (loc, c) in d.items():
        if c != '.':
            if c not in cm:
                cm[c] = []
            cm[c].append(loc)
    return cm

In [ ]:
def q_1(data):
    m = parse_data(data)
    cm = build_char_map(m)
    i_max = max(c.imag for c in m)
    j_max = max(c.real for c in m)

    antinodes = set()
    for char, locs in cm.items():
        for loc_a, loc_b in itertools.combinations(locs, 2):
            antinode_ab = 2 * loc_a - loc_b
            antinode_ba = 2 * loc_b - loc_a
            for an in [antinode_ab, antinode_ba]:
                if (0 <= an.imag <= i_max) and (0 <= an.real <= j_max):
                    antinodes.add(an)

    return len(antinodes)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 14
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
from fractions import Fraction

def resolve_antinode_step_size(loc_a: complex, loc_b: complex) -> complex:
    rise = loc_b.imag - loc_a.imag
    run = loc_b.real - loc_a.real
    slope = Fraction(int(rise), int(run))
    return slope.denominator + slope.numerator * 1j

assert resolve_antinode_step_size(0, 1 + 1j) == 1 + 1j
assert resolve_antinode_step_size(0, 1 + 2j) == 1 + 2j
assert resolve_antinode_step_size(0, 2 + 4j) == 1 + 2j
assert resolve_antinode_step_size(1 - 1j, 2 + 4j) == 1 + 5j
assert resolve_antinode_step_size(1 - 1j, 3 + 5j) == 1 + 3j
assert resolve_antinode_step_size(0, 3 + 1j) == 3 + 1j

In [ ]:
def is_in_range(z: complex, i_max: int, j_max: int) -> bool:
    return (0 <= z.imag <= i_max) and (0 <= z.real <= j_max)

In [ ]:
def q_2(data):
    m = parse_data(data)
    cm = build_char_map(m)
    i_max = max(c.imag for c in m)
    j_max = max(c.real for c in m)

    antinodes = set()
    for char, locs in cm.items():
        for loc_a, loc_b in itertools.combinations(locs, 2):
            # LOGGER.info(f"{loc_a = }, {loc_b = }")
            # start at a, walk step sizes towards and then away from b until you leave the grid
            step = resolve_antinode_step_size(loc_a, loc_b)
            # LOGGER.info(f"{step = }")
            for dir_mod in [1, -1]:
                # LOGGER.info(f"{dir_mod = }")
                z = loc_a
                antinodes.add(z)
                # LOGGER.debug(f"{z = }")
                while True:
                    z += dir_mod * step
                    # LOGGER.debug(f"{z = } after step")
                    if is_in_range(z, i_max, j_max):
                        antinodes.add(z)
                        # LOGGER.debug(antinodes)
                    else:
                        break

    return len(antinodes)

#### tests

In [ ]:
new_test_data = """T.........
...T......
.T........
..........
..........
..........
..........
..........
..........
.........."""

new_test_data_answer = """
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
"""

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    v = q_2(new_test_data)
    assert v == 9, v
    v = q_2(test_data)
    assert v == 34, v
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin